In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import namedtuple
import random
from random import sample
from collections import Counter
import numba
import os
%matplotlib inline

In [23]:
from xgboost import XGBRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt 
import xgboost

In [24]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [25]:
path2021 = "/home/pratik/Documents/MSc/Project/Data Preparation/data/2020-21/players/"
path2122 = "/home/pratik/Documents/MSc/Project/Data Preparation/data/2021-22/players/"
dir_list_2021 = os.listdir(path2021)
dir_list_2122 = os.listdir(path2122)

In [26]:
column_names = ["player_name"]
for i in range(25):
    if i+1 < 10:
        col = "gw_0"+str(i+1)+"_pred"
    else:
        col = "gw_"+str(i+1)+"_pred"
    column_names.append(col)
column_names.append("total_predicted")
column_names.append("actual_points")
column_names.append("rmse")
df = pd.DataFrame(columns = column_names)
df.head()

,player_name,gw_01_pred,gw_02_pred,gw_03_pred,gw_04_pred,gw_05_pred,gw_06_pred,gw_07_pred,gw_08_pred,gw_09_pred,gw_10_pred,gw_11_pred,gw_12_pred,gw_13_pred,gw_14_pred,gw_15_pred,gw_16_pred,gw_17_pred,gw_18_pred,gw_19_pred,gw_20_pred,gw_21_pred,gw_22_pred,gw_23_pred,gw_24_pred,gw_25_pred,total_predicted,actual_points,rmse


In [27]:
params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}

In [28]:
data = []
# for i in range(149, len(dir_list_2122)):
for i in range(len(dir_list_2122)):
    print("CURRENT PLAYER: ", dir_list_2122[i])
    row_data = []
    data_test = pd.read_csv(path2122 + str(dir_list_2122[i]) + "/gw.csv")
    if dir_list_2122[i] in dir_list_2021:
        data_train = pd.read_csv(path2021 + str(dir_list_2021[i]) + "/gw.csv")
        
        data_train = data_train.drop(data_train.loc[:, 'assists':'goals_scored'].columns, axis = 1)
        data_train = data_train.drop(data_train.loc[:, 'ict_index':'red_cards'].columns, axis = 1)
        data_train = data_train.drop(['saves'], axis = 1)
        data_train = data_train.drop(['goal_difference'], axis = 1)
        data_train = data_train.drop(data_train.loc[:, 'team_a_score':'threat'].columns, axis = 1)
        data_train = data_train.drop(data_train.loc[:, 'transfers_in':'transfers_out'].columns, axis = 1)
        data_train = data_train.drop(data_train.loc[:, 'was_home':'opp_team'].columns, axis = 1)
        data_train = data_train.drop(['Position','round'], axis = 1)
#         data_train = data_train.drop(data_train.loc[:, 'Season_1819':'Season_2021'].columns, axis = 1)
        
        data_test = data_test.drop(data_test.loc[:, 'assists':'goals_scored'].columns, axis = 1)
        data_test = data_test.drop(data_test.loc[:, 'ict_index':'red_cards'].columns, axis = 1)
        data_test = data_test.drop(['saves'], axis = 1)
        data_test = data_test.drop(['goal_difference'], axis = 1)
        data_test = data_test.drop(data_test.loc[:, 'team_a_score':'threat'].columns, axis = 1)
        data_test = data_test.drop(data_test.loc[:, 'transfers_in':'transfers_out'].columns, axis = 1)
        data_test = data_test.drop(data_test.loc[:, 'was_home':'opp_team'].columns, axis = 1)
        data_test = data_test.drop(['Position','round'], axis = 1)
#         data_test = data_test.drop(['Season_2122'], axis = 1)
        
        var_columns = [c for c in data_train.columns if c not in ['total_points','Season_1819','Season_1920','Season_2021','round']]
        X_train = data_train.loc[:, var_columns]
        Y_train = data_train.loc[:, 'total_points']

        var_columns_test = [c for c in data_test.columns if c not in ['total_points','Season_2122','round']]
        X_test = data_test.loc[:, var_columns_test]
        Y_test = data_test.loc[:, 'total_points']
        
        xgb_reg = XGBRegressor()
#         random_search = RandomizedSearchCV(xgb_reg,param_distributions=params,n_iter=5,n_jobs=-1,cv=5,verbose=3)
        
#         random_search.fit(X_train,Y_train)
#         tuned_params = random_search.best_params_
        
#         xgb_reg = XGBRegressor(min_child_weight = tuned_params['min_child_weight'],
#                               max_depth = tuned_params['max_depth'],
#                               learning_rate = tuned_params['learning_rate'],
#                               gamma = tuned_params['gamma'],
#                               colsample_bytree = tuned_params['colsample_bytree'])
        
        xgb_reg.fit(X_train, Y_train)
        Y_pred = xgb_reg.predict(X_test)
        
        sum_pred = 0
        sum_test = 0
        for j in range(len(Y_pred)):
            Y_pred[j] = round(Y_pred[j])
            sum_pred += Y_pred[j]
            sum_test += Y_test[j]
            Y_pred[j] = round(Y_pred[j])
        
        mse = mean_squared_error(Y_test, Y_pred)
        rmse = np.sqrt(mse)
        
        for j in range(len(column_names)):
            if column_names[j] == "player_name":
                row_data.append(dir_list_2122[i])
            elif column_names[j] == "total_predicted":
                row_data.append(sum_pred)
            elif column_names[j] == "actual_points":
                row_data.append(sum_test)
            elif column_names[j] == "rmse":
                row_data.append(rmse)
            else:
                try:
                    row_data.append(Y_pred[int(column_names[j][3:5]) - 1])
                except:
                    row_data.append("NP")
        del xgb_reg
        data.append(row_data)
                     
    else:
        sum_test = 0
        for j in range(len(data_test)):
            sum_test += data_test['total_points'][j]
            
        for j in range(len(column_names)):
            if column_names[j] == "player_name":
                row_data.append(dir_list_2122[i])
            elif column_names[j] == "total_predicted":
                row_data.append("NA")
            elif column_names[j] == "actual_points":
                row_data.append(sum_test)
            elif column_names[j] == "rmse":
                row_data.append("NA")
            else:
                row_data.append("NA")

        data.append(row_data)
    print("COMPLETED ", i+1, " Out of ", len(dir_list_2122), " Players List")
    print("__________________________________________________________________")
    
    
    df = pd.DataFrame(data, columns = column_names)   
    df.to_csv( "/home/pratik/Documents/MSc/Project/Data Preparation/No Tuning/predictions.csv", index=False)
#     df.to_csv( "/home/pratik/Documents/MSc/Project/Data Preparation/predictions3.csv", index=False)

CURRENT PLAYER:  Adam Masina
COMPLETED  1  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Tyrick Mitchell
COMPLETED  2  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Filip Marschall
COMPLETED  3  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Alex Nicolao Telles
COMPLETED  4  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Zack Steffen
COMPLETED  5  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Sam Byram
COMPLETED  6  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Timothy Castagne
COMPLETED  7  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Nohan Kenne

COMPLETED  60  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Gabriel Fernando de Jesus
COMPLETED  61  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Paul Pogba
COMPLETED  62  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Bali Mumba
COMPLETED  63  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Lamare Bogarde
COMPLETED  64  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Lewis Richardson
COMPLETED  65  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Yves Bissouma
COMPLETED  66  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Anthony Martial
COMPLETED  67  

COMPLETED  120  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Emmanuel Dennis
COMPLETED  121  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Angus Gunn
COMPLETED  122  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Aaron Wan-Bissaka
COMPLETED  123  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Kieran Trippier
COMPLETED  124  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ivan Toney
COMPLETED  125  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Hassane Kamara
COMPLETED  126  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ashley Barnes
COMPLETED  127  O

COMPLETED  177  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Yerson Mosquera Valdelamar
COMPLETED  178  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Andy Lonergan
COMPLETED  179  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Harvey White
COMPLETED  180  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Michael Olise
COMPLETED  181  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Erik Lamela
COMPLETED  182  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Takumi Minamino
COMPLETED  183  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Adam Lallana
COMPLETED

COMPLETED  236  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Bernard Anício Caldeira Duarte
COMPLETED  237  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Toby Alderweireld
COMPLETED  238  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Winston Reid
COMPLETED  239  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Granit Xhaka
COMPLETED  240  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Crysencio Summerville
COMPLETED  241  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Tino Livramento
COMPLETED  242  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ibrah

...

COMPLETED  469  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Rico Henry
COMPLETED  470  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Samuel Kalu
COMPLETED  471  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Virgil van Dijk
COMPLETED  472  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Antonio Rüdiger
COMPLETED  473  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Danny Welbeck
COMPLETED  474  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Nathan Redmond
COMPLETED  475  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  João Virgínia
COMPLETED  476  Out 

COMPLETED  527  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Rhys Williams
COMPLETED  528  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Mohamed Naser El Sayed Elneny
COMPLETED  529  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Federico Fernández
COMPLETED  530  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ferran Torres
COMPLETED  531  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Bryan Gil Salvatierra
COMPLETED  532  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Raúl Jiménez
COMPLETED  533  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Remi M

COMPLETED  587  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Vladimir Coufal
COMPLETED  588  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Gabriel Teodoro Martinelli Silva
COMPLETED  589  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Kurt Zouma
COMPLETED  590  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Stuart McKinstry
COMPLETED  591  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ross Barkley
COMPLETED  592  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Kjell Scherpen
COMPLETED  593  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Mbwana Ally S

COMPLETED  645  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ezri Konsa Ngoyo
COMPLETED  646  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Héctor Bellerín
COMPLETED  647  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Bailey Peacock-Farrell
COMPLETED  648  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Matt Doherty
COMPLETED  649  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Jakub Moder
COMPLETED  650  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Anthony Elanga
COMPLETED  651  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Christian Pulisic
COMP

COMPLETED  703  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  William Troost-Ekong
COMPLETED  704  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Lewis Dunk
COMPLETED  705  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Jarosław Jach
COMPLETED  706  Out of  707  Players List
__________________________________________________________________
CURRENT PLAYER:  Ajibola Alese
COMPLETED  707  Out of  707  Players List
__________________________________________________________________
